In [1]:
import pandas as pd
import time
import random
import requests
import itertools
import copy
import numpy as np
# Retrieve the S&P 500 stock symbols and sector data
def get_sp500_symbols_and_sectors():
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    tables = pd.read_html(url)
    sp500_df = tables[0]  # First table contains stock symbols and sectors
    return sp500_df[['Symbol', 'GICS Sector']].rename(columns={'Symbol': 'ticker', 'GICS Sector': 'sector'})

# Find all tickers in the same sector as the given ticker
def get_same_sector_tickers(ticker, sp500_df):
    sector = sp500_df.loc[sp500_df['ticker'] == ticker, 'sector'].values
    if len(sector) == 0:
        print(f"❌ {ticker} not found in S&P 500 list.")
        return []
    
    sector = sector[0]
    same_sector_tickers = sp500_df[sp500_df['sector'] == sector]['ticker'].tolist()
    print(f"  Sector: {sector}")
    print(f"📊 Found {len(same_sector_tickers)} tickers in the same sector as {ticker}")
    
    return same_sector_tickers

In [3]:
sp500_df = get_sp500_symbols_and_sectors()
sector_data = sp500_df.set_index('ticker')['sector'].to_dict()

In [5]:
def get_company_fundamentals(symbols):
    """
    Fetch fundamental financial data for multiple stock symbols using Financial Modeling Prep (FMP).
    If primary data is missing, try alternative endpoints.
    
    :param symbols: List of stock ticker symbols.
    :return: Dictionary with fundamental metrics for each stock.
    """
    fundamentals_data = {} 
    
    API_KEY_li = ['rEwY9uKTA66xG5uK8CAifKsu9tSAzuHd','oFLf4RZvkY3ksOBQ71PM39Cv84WOqqyV']
    numbers = list(range(len(API_KEY_li)))
    cycler = itertools.cycle(numbers)
    API_KEY = API_KEY_li[next(cycler)]
    used_all_api_keys = set()
    
    for symbol in symbols:
        attempts = 0
        wait_time = 5  # Start with 5-second wait
        
        while attempts < 3:  # Retry up to 3 times if rate-limited
            try:
                # Company Profile
                url_profile = f"https://financialmodelingprep.com/api/v3/profile/{symbol}?apikey={API_KEY}"
                response = requests.get(url_profile)
                if response.status_code == 200:
                    data = response.json()
                    if not data or data == []:
                        data = [{}]
                        attempts += 1
                        print(f"⚠️ No company profile value for {symbol}.")

                elif response.status_code == 429:  # Rate limit hit
                    print(f"⚠️ Rate limit hit for {symbol}.")
                    time.sleep(60)

                    #sleep and try again
                    response = requests.get(url_profile)
                    if response.status_code == 200:
                        data = response.json()
                        if not data or data == []:
                            data = [{}]
                            attempts += 1
                            print(f"⚠️ No company profile value for {symbol}.")

                    elif response.status_code == 429:  
                        print(f"⚠️ Need to change api for {symbol}.")
                        used_all_api_keys.add(API_KEY)
                        if len(used_all_api_keys) == len(API_KEY_li):
                            print("🔄 All API keys have been used. Pausing the program.")
                            return fundamentals_data
                        else:
                            API_KEY = API_KEY_li[next(cycler)]
                            print(API_KEY)
                            url_profile = f"https://financialmodelingprep.com/api/v3/profile/{symbol}?apikey={API_KEY}"
                            response = requests.get(url_profile)
                            data = response.json()
                            if not data or data == []:
                                data = [{}]
                                attempts += 1
                                print(f"⚠️ No company profile value for {symbol}.")
                    else:
                        raise ValueError(f"Unexpected API response: {response.text}")
                else:
                    raise ValueError(f"Unexpected API response: {response.text}")
                
                company_data = data[0]  

                #Key Metrics
                url_metrics = f"https://financialmodelingprep.com/api/v3/key-metrics/{symbol}?apikey={API_KEY}"
                response = requests.get(url_metrics)
                if response.status_code == 200:
                    data = response.json()
                    if not data or data == []:
                        data = [{}]
                        attempts += 1
                        print(f"⚠️ No key metrics value for {symbol}.")

                elif response.status_code == 429:  # Rate limit hit
                    print(f"⚠️ Rate limit hit for {symbol}.")
                    time.sleep(60)

                    #sleep and try again
                    response = requests.get(url_metrics)
                    if response.status_code == 200:
                        data = response.json()
                        if not data or data == []:
                            data = [{}]
                            attempts += 1
                            print(f"⚠️ No key metrics value for {symbol}.")

                    elif response.status_code == 429:  
                        print(f"⚠️ Need to change api for {symbol}.")
                        used_all_api_keys.add(API_KEY)
                        if len(used_all_api_keys) == len(API_KEY_li):
                            print("🔄 All API keys have been used. Pausing the program.")
                            return fundamentals_data
                        else:
                            API_KEY = API_KEY_li[next(cycler)]
                            print(API_KEY)
                            url_metrics = f"https://financialmodelingprep.com/api/v3/key-metrics/{symbol}?apikey={API_KEY}"
                            response = requests.get(url_metrics)
                            data = response.json()
                            if not data or data == []:
                                data = [{}]
                                attempts += 1
                                print(f"⚠️ No key metrics value for {symbol}.")
                    else:
                        raise ValueError(f"Unexpected API response: {response.text}")
                else:
                    raise ValueError(f"Unexpected API response: {response.text}")                
            
                metrics_data = data[0]  

                #ratios
                url_ratios = f"https://financialmodelingprep.com/api/v3/ratios/{symbol}?apikey={API_KEY}"
                response = requests.get(url_ratios)
                if response.status_code == 200:
                    data = response.json()
                    if not data or data == []:
                        data = [{}]
                        attempts += 1
                        print(f"⚠️ No ratios value for {symbol}.")

                elif response.status_code == 429:  # Rate limit hit
                    print(f"⚠️ Rate limit hit for {symbol}.")
                    time.sleep(60)

                    #sleep and try again
                    response = requests.get(url_ratios)
                    if response.status_code == 200:
                        data = response.json()
                        if not data or data == []:
                            data = [{}]
                            attempts += 1
                            print(f"⚠️ No ratios value for {symbol}.")
                    
                    #change api and try again
                    elif response.status_code == 429:  
                        print(f"⚠️ Need to change api for {symbol}.")
                        used_all_api_keys.add(API_KEY)
                        if len(used_all_api_keys) == len(API_KEY_li):
                            print("🔄 All API keys have been used. Pausing the program.")
                            return fundamentals_data
                        else:
                            API_KEY = API_KEY_li[next(cycler)]
                            print(API_KEY)
                            url_ratios = f"https://financialmodelingprep.com/api/v3/ratios/{symbol}?apikey={API_KEY}"
                            response = requests.get(url_ratios)
                            data = response.json()
                            if not data or data == []:
                                data = [{}]
                                attempts += 1
                                print(f"⚠️ No ratios value for {symbol}.")
                    else:
                        raise ValueError(f"Unexpected API response: {response.text}")
                else:
                    raise ValueError(f"Unexpected API response: {response.text}")                 
                
                ratios_data = data[0]  
                
                
                # Dividends
                url_Dividends = f"https://financialmodelingprep.com/api/v3/historical-price-full/stock_dividend/{symbol}?apikey={API_KEY}"
                response = requests.get(url_Dividends)
                if response.status_code == 200:
                    data = response.json()
                    if not data or data == {}:
                        data = [{}]
                        attempts += 1
                        print(f"⚠️ No dividends value for {symbol}.")

                elif response.status_code == 429:  # Rate limit hit
                    print(f"⚠️ Rate limit hit for {symbol}.")
                    time.sleep(60)

                    #sleep and try again
                    response = requests.get(url_Dividends)
                    if response.status_code == 200:
                        data = response.json()
                        if not data or data == {}:
                            data = [{}]
                            attempts += 1
                            print(f"⚠️ No dividends value for {symbol}.")

                    elif response.status_code == 429:  
                        print(f"⚠️ Need to change api for {symbol}.")
                        used_all_api_keys.add(API_KEY)
                        if len(used_all_api_keys) == len(API_KEY_li):
                            print("🔄 All API keys have been used. Pausing the program.")
                            return fundamentals_data
                        else:
                            API_KEY = API_KEY_li[next(cycler)]
                            print(API_KEY)
                            url_Dividends = f"https://financialmodelingprep.com/api/v3/historical-price-full/stock_dividend/{symbol}?apikey={API_KEY}"
                            response = requests.get(url_Dividends)
                            data = response.json()
                            if not data or data == {}:
                                data = [{}]
                                attempts += 1
                                print(f"⚠️ No dividends value for {symbol}.")
                    else:
                        raise ValueError(f"Unexpected API response: {response.text}")
                else:
                    raise ValueError(f"Unexpected API response: {response.text}")    
                
                
                if data.get('historical', [{}]) == []:
                    Dividends_data = {}
                else:
                    Dividends_data = data.get('historical', [{}])[0]#dividends returns dict
                   
                #balance sheet
                url_balance = f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{symbol}?apikey={API_KEY}"
                response = requests.get(url_balance)
                if response.status_code == 200:
                    data = response.json()
                    if not data or data == []:
                        data = [{}]
                        attempts += 1
                        print(f"⚠️ No dividends value for {symbol}.")

                elif response.status_code == 429:  # Rate limit hit
                    print(f"⚠️ Rate limit hit for {symbol}.")
                    time.sleep(60)

                    #sleep and try again
                    response = requests.get(url_balance)
                    if response.status_code == 200:
                        data = response.json()
                        if not data or data == []:
                            data = [{}]
                            attempts += 1
                            print(f"⚠️ No dividends value for {symbol}.")

                    elif response.status_code == 429:  
                        print(f"⚠️ Need to change api for {symbol}.")
                        used_all_api_keys.add(API_KEY)
                        if len(used_all_api_keys) == len(API_KEY_li):
                            print("🔄 All API keys have been used. Pausing the program.")
                            return fundamentals_data
                        else:
                            API_KEY = API_KEY_li[next(cycler)]
                            print(API_KEY)
                            url_balance = f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{symbol}?apikey={API_KEY}"
                            response = requests.get(url_balance)
                            data = response.json()
                            if not data or data == []:
                                data = [{}]
                                attempts += 1
                                print(f"⚠️ No dividends value for {symbol}.")
                    else:
                        raise ValueError(f"Unexpected API response: {response.text}")
                else:
                    raise ValueError(f"Unexpected API response: {response.text}")                    
                
                balance_data = data[0]  
                    
                #cash flow statement
                url_cash = f"https://financialmodelingprep.com/api/v3/cash-flow-statement/{symbol}?apikey={API_KEY}"
                response = requests.get(url_cash)
                if response.status_code == 200:
                    data = response.json()
                    if not data or data == []:
                        data = [{}]
                        attempts += 1
                        print(f"⚠️ No cash flow value for {symbol}.")

                elif response.status_code == 429:  # Rate limit hit
                    print(f"⚠️ Rate limit hit for {symbol}.")
                    time.sleep(60)

                    #sleep and try again
                    response = requests.get(url_cash)
                    if response.status_code == 200:
                        data = response.json()
                        if not data or data == []:
                            data = [{}]
                            attempts += 1
                            print(f"⚠️ No cash flow value for {symbol}.")

                    elif response.status_code == 429:  
                        print(f"⚠️ Need to change api for {symbol}.")
                        used_all_api_keys.add(API_KEY)
                        if len(used_all_api_keys) == len(API_KEY_li):
                            print("🔄 All API keys have been used. Pausing the program.")
                            return fundamentals_data
                        else:
                            API_KEY = API_KEY_li[next(cycler)]
                            print(API_KEY)
                            url_cash = f"https://financialmodelingprep.com/api/v3/cash-flow-statement/{symbol}?apikey={API_KEY}"
                            response = requests.get(url_cash)
                            data = response.json()
                            if not data or data == []:
                                data = [{}]
                                attempts += 1
                                print(f"⚠️ No cash flow value for {symbol}.")
                    else:
                        raise ValueError(f"Unexpected API response: {response.text}")
                else:
                    raise ValueError(f"Unexpected API response: {response.text}")               

                cash_data = data[0]  

                # Extract relevant metrics (fill missing values with alternative sources)
                metrics = {
                    'P/E Ratio': company_data.get('peRatio') or ratios_data.get('priceEarningsRatio', None),
                    'Market Cap': company_data.get('mktCap', None),
                    'Revenue': company_data.get('revenue') or metrics_data.get('revenuePerShare', None),
                    'Beta': company_data.get('beta', None),
                    'Return on Assets (ROA)': company_data.get('returnOnAssets') or ratios_data.get('returnOnAssets', None),
                    'Return on Equity (ROE)': company_data.get('returnOnEquity') or ratios_data.get('returnOnEquity', None),
                    'dividendYield':ratios_data.get('dividendYield') or metrics_data.get('dividendYield', None),
                    'dividend per share':Dividends_data.get('adjDividend', None),
                    'total debt':balance_data.get('totalDebt', None),
                    'total cash':balance_data.get('cashAndCashEquivalents', None),
                    'free cash flow':cash_data.get('freeCashFlow', None),
                    'operating cash flow':cash_data.get('operatingCashFlow', None),
                    'net income':cash_data.get('netIncome', None)
                }

                fundamentals_data[symbol] = metrics
                print(f"  Retrieved fundamentals for {symbol}")
                break  # Exit retry loop if successful

            except Exception as e:
                print(f"⚠️ Attempt {attempts + 1}: Failed to retrieve data for {symbol}. Error: {e}")
                attempts += 1
                time.sleep(wait_time)
                wait_time *= 2  # Exponential backoff
    
                if attempts == 3:  # Skip stock if it fails after 3 tries
                    print(f"❌ Skipping {symbol} after 3 failed attempts.")
                    fundamentals_data[symbol] = None

    return fundamentals_data

In [7]:
def get_industry_avg(stocks_data, sector_data):
    # Step 1: Add sector information to stocks_data
    for ticker, data in stocks_data.items():
        if ticker in sector_data:
            data['sector'] = sector_data[ticker]
        else:
            data['sector'] = None
    
    # Step 2: Group stocks by sector
    sector_groups = {}
    first_key_value = next(iter(stocks_data.values()))
    keys_list = [key for key in first_key_value.keys() if key != 'sector']
    
    for symbol, data in stocks_data.items():
        sector = data['sector']
        if sector not in sector_groups:
            sector_groups[sector] = []
        sector_groups[sector].append((symbol, data))

    # Step 3: Calculate sector averages for each metric
    for sector, stock_list in sector_groups.items():
        sector_avg = {}  # Dictionary to store sector averages

        for metric in keys_list:
            values = [stock[1].get(metric) for stock in stock_list if stock[1].get(metric) is not None]
            # Ensure only numerical values are considered
            values = [v for v in values if isinstance(v, (int, float))]
            
            if values:
                sector_avg[f'{metric}_avg'] = np.mean(values)
            else:
                sector_avg[f'{metric}_avg'] = None  

        # Step 4: Update each stock with the sector averages
        for symbol, data in stock_list:
            data.update(sector_avg)

    return stocks_data

In [9]:
tickers = sp500_df["ticker"].tolist()
fundamentals_data = get_company_fundamentals(tickers)
new_data = copy.deepcopy(fundamentals_data)

  Retrieved fundamentals for MMM
  Retrieved fundamentals for AOS
  Retrieved fundamentals for ABT
  Retrieved fundamentals for ABBV
  Retrieved fundamentals for ACN
  Retrieved fundamentals for ADBE
  Retrieved fundamentals for AMD
  Retrieved fundamentals for AES
  Retrieved fundamentals for AFL
  Retrieved fundamentals for A
  Retrieved fundamentals for APD
  Retrieved fundamentals for ABNB
  Retrieved fundamentals for AKAM
  Retrieved fundamentals for ALB
  Retrieved fundamentals for ARE
  Retrieved fundamentals for ALGN
  Retrieved fundamentals for ALLE
  Retrieved fundamentals for LNT
⚠️ Rate limit hit for ALL.
⚠️ Need to change api for ALL.
oFLf4RZvkY3ksOBQ71PM39Cv84WOqqyV
  Retrieved fundamentals for ALL
  Retrieved fundamentals for GOOGL
  Retrieved fundamentals for GOOG
  Retrieved fundamentals for MO
  Retrieved fundamentals for AMZN
  Retrieved fundamentals for AMCR
  Retrieved fundamentals for AEE
  Retrieved fundamentals for AEP
  Retrieved fundamentals for AXP
  Retrieve

In [17]:
tickers[59:]

['BDX',
 'BRK.B',
 'BBY',
 'TECH',
 'BIIB',
 'BLK',
 'BX',
 'BK',
 'BA',
 'BKNG',
 'BWA',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BRO',
 'BF.B',
 'BLDR',
 'BG',
 'BXP',
 'CHRW',
 'CDNS',
 'CZR',
 'CPT',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'COR',
 'CNC',
 'CNP',
 'CF',
 'CRL',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA',
 'CAG',
 'COP',
 'ED',
 'STZ',
 'CEG',
 'COO',
 'CPRT',
 'GLW',
 'CPAY',
 'CTVA',
 'CSGP',
 'COST',
 'CTRA',
 'CRWD',
 'CCI',
 'CSX',
 'CMI',
 'CVS',
 'DHR',
 'DRI',
 'DVA',
 'DAY',
 'DECK',
 'DE',
 'DELL',
 'DAL',
 'DVN',
 'DXCM',
 'FANG',
 'DLR',
 'DFS',
 'DG',
 'DLTR',
 'D',
 'DPZ',
 'DOV',
 'DOW',
 'DHI',
 'DTE',
 'DUK',
 'DD',
 'EMN',
 'ETN',
 'EBAY',
 'ECL',
 'EIX',
 'EW',
 'EA',
 'ELV',
 'EMR',
 'ENPH',
 'ETR',
 'EOG',
 'EPAM',
 'EQT',
 'EFX',
 'EQIX',
 'EQR',
 'ERIE',
 'ESS',
 'EL',
 'EG',
 'EVRG',
 'ES',
 

In [19]:
def get_company_fundamentals(symbols,API_KEY_li):
    """
    Fetch fundamental financial data for multiple stock symbols using Financial Modeling Prep (FMP).
    If primary data is missing, try alternative endpoints.
    
    :param symbols: List of stock ticker symbols.
    :return: Dictionary with fundamental metrics for each stock.
    """
    fundamentals_data = {} 
    
    numbers = list(range(len(API_KEY_li)))
    cycler = itertools.cycle(numbers)
    API_KEY = API_KEY_li[next(cycler)]
    used_all_api_keys = set()
    
    for symbol in symbols:
        attempts = 0
        wait_time = 5  # Start with 5-second wait
        
        while attempts < 3:  # Retry up to 3 times if rate-limited
            try:
                # Company Profile
                url_profile = f"https://financialmodelingprep.com/api/v3/profile/{symbol}?apikey={API_KEY}"
                response = requests.get(url_profile)
                if response.status_code == 200:
                    data = response.json()
                    if not data or data == []:
                        data = [{}]
                        attempts += 1
                        print(f"⚠️ No company profile value for {symbol}.")

                elif response.status_code == 429:  # Rate limit hit
                    print(f"⚠️ Rate limit hit for {symbol}.")
                    time.sleep(60)

                    #sleep and try again
                    response = requests.get(url_profile)
                    if response.status_code == 200:
                        data = response.json()
                        if not data or data == []:
                            data = [{}]
                            attempts += 1
                            print(f"⚠️ No company profile value for {symbol}.")

                    elif response.status_code == 429:  
                        print(f"⚠️ Need to change api for {symbol}.")
                        used_all_api_keys.add(API_KEY)
                        if len(used_all_api_keys) == len(API_KEY_li):
                            print("🔄 All API keys have been used. Pausing the program.")
                            return fundamentals_data
                        else:
                            API_KEY = API_KEY_li[next(cycler)]
                            print(API_KEY)
                            url_profile = f"https://financialmodelingprep.com/api/v3/profile/{symbol}?apikey={API_KEY}"
                            response = requests.get(url_profile)
                            data = response.json()
                            if not data or data == []:
                                data = [{}]
                                attempts += 1
                                print(f"⚠️ No company profile value for {symbol}.")
                    else:
                        raise ValueError(f"Unexpected API response: {response.text}")
                else:
                    raise ValueError(f"Unexpected API response: {response.text}")
                
                company_data = data[0]  

                #Key Metrics
                url_metrics = f"https://financialmodelingprep.com/api/v3/key-metrics/{symbol}?apikey={API_KEY}"
                response = requests.get(url_metrics)
                if response.status_code == 200:
                    data = response.json()
                    if not data or data == []:
                        data = [{}]
                        attempts += 1
                        print(f"⚠️ No key metrics value for {symbol}.")

                elif response.status_code == 429:  # Rate limit hit
                    print(f"⚠️ Rate limit hit for {symbol}.")
                    time.sleep(60)

                    #sleep and try again
                    response = requests.get(url_metrics)
                    if response.status_code == 200:
                        data = response.json()
                        if not data or data == []:
                            data = [{}]
                            attempts += 1
                            print(f"⚠️ No key metrics value for {symbol}.")

                    elif response.status_code == 429:  
                        print(f"⚠️ Need to change api for {symbol}.")
                        used_all_api_keys.add(API_KEY)
                        if len(used_all_api_keys) == len(API_KEY_li):
                            print("🔄 All API keys have been used. Pausing the program.")
                            return fundamentals_data
                        else:
                            API_KEY = API_KEY_li[next(cycler)]
                            print(API_KEY)
                            url_metrics = f"https://financialmodelingprep.com/api/v3/key-metrics/{symbol}?apikey={API_KEY}"
                            response = requests.get(url_metrics)
                            data = response.json()
                            if not data or data == []:
                                data = [{}]
                                attempts += 1
                                print(f"⚠️ No key metrics value for {symbol}.")
                    else:
                        raise ValueError(f"Unexpected API response: {response.text}")
                else:
                    raise ValueError(f"Unexpected API response: {response.text}")                
            
                metrics_data = data[0]  

                #ratios
                url_ratios = f"https://financialmodelingprep.com/api/v3/ratios/{symbol}?apikey={API_KEY}"
                response = requests.get(url_ratios)
                if response.status_code == 200:
                    data = response.json()
                    if not data or data == []:
                        data = [{}]
                        attempts += 1
                        print(f"⚠️ No ratios value for {symbol}.")

                elif response.status_code == 429:  # Rate limit hit
                    print(f"⚠️ Rate limit hit for {symbol}.")
                    time.sleep(60)

                    #sleep and try again
                    response = requests.get(url_ratios)
                    if response.status_code == 200:
                        data = response.json()
                        if not data or data == []:
                            data = [{}]
                            attempts += 1
                            print(f"⚠️ No ratios value for {symbol}.")
                    
                    #change api and try again
                    elif response.status_code == 429:  
                        print(f"⚠️ Need to change api for {symbol}.")
                        used_all_api_keys.add(API_KEY)
                        if len(used_all_api_keys) == len(API_KEY_li):
                            print("🔄 All API keys have been used. Pausing the program.")
                            return fundamentals_data
                        else:
                            API_KEY = API_KEY_li[next(cycler)]
                            print(API_KEY)
                            url_ratios = f"https://financialmodelingprep.com/api/v3/ratios/{symbol}?apikey={API_KEY}"
                            response = requests.get(url_ratios)
                            data = response.json()
                            if not data or data == []:
                                data = [{}]
                                attempts += 1
                                print(f"⚠️ No ratios value for {symbol}.")
                    else:
                        raise ValueError(f"Unexpected API response: {response.text}")
                else:
                    raise ValueError(f"Unexpected API response: {response.text}")                 
                
                ratios_data = data[0]  
                
                
                # Dividends
                url_Dividends = f"https://financialmodelingprep.com/api/v3/historical-price-full/stock_dividend/{symbol}?apikey={API_KEY}"
                response = requests.get(url_Dividends)
                if response.status_code == 200:
                    data = response.json()
                    if not data or data == {}:
                        data = [{}]
                        attempts += 1
                        print(f"⚠️ No dividends value for {symbol}.")

                elif response.status_code == 429:  # Rate limit hit
                    print(f"⚠️ Rate limit hit for {symbol}.")
                    time.sleep(60)

                    #sleep and try again
                    response = requests.get(url_Dividends)
                    if response.status_code == 200:
                        data = response.json()
                        if not data or data == {}:
                            data = [{}]
                            attempts += 1
                            print(f"⚠️ No dividends value for {symbol}.")

                    elif response.status_code == 429:  
                        print(f"⚠️ Need to change api for {symbol}.")
                        used_all_api_keys.add(API_KEY)
                        if len(used_all_api_keys) == len(API_KEY_li):
                            print("🔄 All API keys have been used. Pausing the program.")
                            return fundamentals_data
                        else:
                            API_KEY = API_KEY_li[next(cycler)]
                            print(API_KEY)
                            url_Dividends = f"https://financialmodelingprep.com/api/v3/historical-price-full/stock_dividend/{symbol}?apikey={API_KEY}"
                            response = requests.get(url_Dividends)
                            data = response.json()
                            if not data or data == {}:
                                data = [{}]
                                attempts += 1
                                print(f"⚠️ No dividends value for {symbol}.")
                    else:
                        raise ValueError(f"Unexpected API response: {response.text}")
                else:
                    raise ValueError(f"Unexpected API response: {response.text}")    
                
                
                if data.get('historical', [{}]) == []:
                    Dividends_data = {}
                else:
                    Dividends_data = data.get('historical', [{}])[0]#dividends returns dict
                   
                #balance sheet
                url_balance = f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{symbol}?apikey={API_KEY}"
                response = requests.get(url_balance)
                if response.status_code == 200:
                    data = response.json()
                    if not data or data == []:
                        data = [{}]
                        attempts += 1
                        print(f"⚠️ No dividends value for {symbol}.")

                elif response.status_code == 429:  # Rate limit hit
                    print(f"⚠️ Rate limit hit for {symbol}.")
                    time.sleep(60)

                    #sleep and try again
                    response = requests.get(url_balance)
                    if response.status_code == 200:
                        data = response.json()
                        if not data or data == []:
                            data = [{}]
                            attempts += 1
                            print(f"⚠️ No dividends value for {symbol}.")

                    elif response.status_code == 429:  
                        print(f"⚠️ Need to change api for {symbol}.")
                        used_all_api_keys.add(API_KEY)
                        if len(used_all_api_keys) == len(API_KEY_li):
                            print("🔄 All API keys have been used. Pausing the program.")
                            return fundamentals_data
                        else:
                            API_KEY = API_KEY_li[next(cycler)]
                            print(API_KEY)
                            url_balance = f"https://financialmodelingprep.com/api/v3/balance-sheet-statement/{symbol}?apikey={API_KEY}"
                            response = requests.get(url_balance)
                            data = response.json()
                            if not data or data == []:
                                data = [{}]
                                attempts += 1
                                print(f"⚠️ No dividends value for {symbol}.")
                    else:
                        raise ValueError(f"Unexpected API response: {response.text}")
                else:
                    raise ValueError(f"Unexpected API response: {response.text}")                    
                
                balance_data = data[0]  
                    
                #cash flow statement
                url_cash = f"https://financialmodelingprep.com/api/v3/cash-flow-statement/{symbol}?apikey={API_KEY}"
                response = requests.get(url_cash)
                if response.status_code == 200:
                    data = response.json()
                    if not data or data == []:
                        data = [{}]
                        attempts += 1
                        print(f"⚠️ No cash flow value for {symbol}.")

                elif response.status_code == 429:  # Rate limit hit
                    print(f"⚠️ Rate limit hit for {symbol}.")
                    time.sleep(60)

                    #sleep and try again
                    response = requests.get(url_cash)
                    if response.status_code == 200:
                        data = response.json()
                        if not data or data == []:
                            data = [{}]
                            attempts += 1
                            print(f"⚠️ No cash flow value for {symbol}.")

                    elif response.status_code == 429:  
                        print(f"⚠️ Need to change api for {symbol}.")
                        used_all_api_keys.add(API_KEY)
                        if len(used_all_api_keys) == len(API_KEY_li):
                            print("🔄 All API keys have been used. Pausing the program.")
                            return fundamentals_data
                        else:
                            API_KEY = API_KEY_li[next(cycler)]
                            print(API_KEY)
                            url_cash = f"https://financialmodelingprep.com/api/v3/cash-flow-statement/{symbol}?apikey={API_KEY}"
                            response = requests.get(url_cash)
                            data = response.json()
                            if not data or data == []:
                                data = [{}]
                                attempts += 1
                                print(f"⚠️ No cash flow value for {symbol}.")
                    else:
                        raise ValueError(f"Unexpected API response: {response.text}")
                else:
                    raise ValueError(f"Unexpected API response: {response.text}")               

                cash_data = data[0]  

                # Extract relevant metrics (fill missing values with alternative sources)
                metrics = {
                    'P/E Ratio': company_data.get('peRatio') or ratios_data.get('priceEarningsRatio', None),
                    'Market Cap': company_data.get('mktCap', None),
                    'Revenue': company_data.get('revenue') or metrics_data.get('revenuePerShare', None),
                    'Beta': company_data.get('beta', None),
                    'Return on Assets (ROA)': company_data.get('returnOnAssets') or ratios_data.get('returnOnAssets', None),
                    'Return on Equity (ROE)': company_data.get('returnOnEquity') or ratios_data.get('returnOnEquity', None),
                    'dividendYield':ratios_data.get('dividendYield') or metrics_data.get('dividendYield', None),
                    'dividend per share':Dividends_data.get('adjDividend', None),
                    'total debt':balance_data.get('totalDebt', None),
                    'total cash':balance_data.get('cashAndCashEquivalents', None),
                    'free cash flow':cash_data.get('freeCashFlow', None),
                    'operating cash flow':cash_data.get('operatingCashFlow', None),
                    'net income':cash_data.get('netIncome', None)
                }

                fundamentals_data[symbol] = metrics
                print(f"  Retrieved fundamentals for {symbol}")
                break  # Exit retry loop if successful

            except Exception as e:
                print(f"⚠️ Attempt {attempts + 1}: Failed to retrieve data for {symbol}. Error: {e}")
                attempts += 1
                time.sleep(wait_time)
                wait_time *= 2  # Exponential backoff
    
                if attempts == 3:  # Skip stock if it fails after 3 tries
                    print(f"❌ Skipping {symbol} after 3 failed attempts.")
                    fundamentals_data[symbol] = None

    return fundamentals_data

In [21]:
API_KEY_li2=["6hY6I1N27gM7IE4suNpVNaUHxBeZw6yc","ug8ggu6nmS3NV6CAjIHD0k1YDeaTIk3w",
    "QaJCrea0m0Jw3X5xNID0b1KKOg3dPUrX", "fMFlUxLxqKwq6qHuYJ9YBIBxwy8drfZM",
    "ujyGSVIOvjWkHMygBvlLziESGhxiUoaG", "CCzHGrEB1dVVVkw11xxwZb6o0UeUu3Yh",
    "pgGOVJAlfchwFlvfcuuAqMukqbPIPK2a", "c8DQ6I283ArGIPx2fxxzG5NOECdB0AzM",
    "h3WqRv1L1aDOlqjTJSOSyTuQOgav3lFZ", "iUwc8g8lNedeWfEg1NwUnvXGM4iIL3ps",
    "cDWmTbAqq82eINliWSFxn4yVv1GbseJH", "IMgmBYeGkVlNgAwdP5xWXqbMNNEewnHl",
    "oDjz2EDlkZhIbdN18m1yOfDxSZkC7i31", "v1s0mDIr0tBLQJi9tf5aHYv6Iqn9EphC",
    "3f08L5h73zYFlEEtw1pg5JKC8qdR7nAR", "rXR0PrHyt4Vcdn1yqWhklKNSZfdGV9qU",
    "EtzquuX82cOSuWHSTqLfS2aP1D3uhLaT", "VrIWXD4KjvZ07IdwmyD81SDJzAjydQtr",
    "WhuS8fEd9aOh7edtwjUBHhznNdc0fdwx", "hPCjYsWS9RDgqCjJCH2mXa3CCk5LR1lu"]
fundamentals_data2 = get_company_fundamentals(tickers[59:],API_KEY_li2)

  Retrieved fundamentals for BDX
⚠️ No company profile value for BRK.B.
⚠️ No key metrics value for BRK.B.
⚠️ No ratios value for BRK.B.
  Retrieved fundamentals for BRK.B
  Retrieved fundamentals for BBY
  Retrieved fundamentals for TECH
  Retrieved fundamentals for BIIB
  Retrieved fundamentals for BLK
  Retrieved fundamentals for BX
  Retrieved fundamentals for BK
  Retrieved fundamentals for BA
  Retrieved fundamentals for BKNG
  Retrieved fundamentals for BWA
  Retrieved fundamentals for BSX
  Retrieved fundamentals for BMY
  Retrieved fundamentals for AVGO
  Retrieved fundamentals for BR
  Retrieved fundamentals for BRO
⚠️ No company profile value for BF.B.
⚠️ No key metrics value for BF.B.
⚠️ No ratios value for BF.B.
⚠️ No dividends value for BF.B.
⚠️ No cash flow value for BF.B.
  Retrieved fundamentals for BF.B
  Retrieved fundamentals for BLDR
  Retrieved fundamentals for BG
  Retrieved fundamentals for BXP
  Retrieved fundamentals for CHRW
  Retrieved fundamentals for CDNS


In [29]:
fundamentals_data.update(fundamentals_data2)

In [33]:
# Import the sqlite3 module, which may be used for database operations later (note: not used in the current code)
import sqlite3

# Import the pandas library for data processing and analysis
import pandas as pd

# Import the time module, typically used for time delays or timing (not used in the current code)
import time

# Define the function get_sp500_symbols_and_sectors to retrieve the S&P 500 component stock symbols and their sectors
def get_sp500_symbols_and_sectors():
    # Specify the target webpage URL, which contains the list of S&P 500 companies
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    
    # Use pandas' read_html method to extract all table data from the webpage; it returns a list of DataFrames
    tables = pd.read_html(url)
    
    # Usually, the first table (index 0) contains the stock symbols, company names, sectors, etc.
    sp500_df = tables[0]
    
    # From the extracted DataFrame, select the 'Symbol' and 'GICS Sector' columns,
    # and rename the columns to 'ticker' (stock symbol) and 'sector' (industry sector) respectively
    return sp500_df[['Symbol', 'GICS Sector']].rename(columns={'Symbol': 'ticker', 'GICS Sector': 'sector'})

# Define the function get_same_sector_tickers, which, given a stock symbol and S&P 500 DataFrame,
# finds all stock symbols in the same sector as the given stock.
def get_same_sector_tickers(ticker, sp500_df):
    # Use loc to filter the DataFrame based on the given ticker, and .values to convert the result to a NumPy array
    sector = sp500_df.loc[sp500_df['ticker'] == ticker, 'sector'].values
    
    # If no sector information is found for the given stock symbol (i.e., the array is empty), print a message and return an empty list
    if len(sector) == 0:
        print(f"❌ {ticker} not found in S&P 500 list.")
        return []
    
    # If found, take the first element of the array as the sector of the stock
    sector = sector[0]
    
    # Filter the DataFrame for all stock symbols that belong to the same sector and convert them to a list
    same_sector_tickers = sp500_df[sp500_df['sector'] == sector]['ticker'].tolist()
    
    # Print the sector information and the number of tickers found in the same sector as the given stock
    print(f"  Sector: {sector}")
    print(f"📊 Found {len(same_sector_tickers)} tickers in the same sector as {ticker}")
    
    # Return the list of stock symbols in the same sector
    return same_sector_tickers


In [35]:
import sqlite3  # Import the sqlite3 module for working with SQLite databases

def update_table_structure(cursor):
    """
    Check if the company_fundamentals table contains new fields.
    If not, add the fields using ALTER TABLE statements.
    The new fields include:
      industry              TEXT,
      dividend_yield        REAL,
      dividend_per_share    REAL,
      total_debt            INTEGER,
      total_cash            INTEGER,
      free_cash_flow        INTEGER,
      operating_cash_flow   INTEGER,
      net_income            INTEGER
    """
    # Query the table structure
    cursor.execute("PRAGMA table_info(company_fundamentals)")
    # Retrieve all column names (the second column in the result)
    existing_columns = [row[1] for row in cursor.fetchall()]

    # Define the new columns and their data types that need to be added
    new_columns = {
        "industry": "TEXT",
        "dividend_yield": "REAL",
        "dividend_per_share": "REAL",
        "total_debt": "INTEGER",
        "total_cash": "INTEGER",
        "free_cash_flow": "INTEGER",
        "operating_cash_flow": "INTEGER",
        "net_income": "INTEGER"
    }

    # For each new column, if it does not exist in the existing columns, add it
    for col, col_type in new_columns.items():
        if col not in existing_columns:
            print(f"Adding column '{col}' to table company_fundamentals...")
            cursor.execute(f"ALTER TABLE company_fundamentals ADD COLUMN {col} {col_type}")

def create_database():
    """
    Create a SQLite database and a table for storing company fundamental data.
    If the table already exists, all data will be deleted and the table structure updated to add new fields.
    """
    # Connect to a SQLite database file named 'company_fundamentals.db'
    conn = sqlite3.connect('company_fundamentals.db')
    cursor = conn.cursor()

    # Check if a table named 'company_fundamentals' already exists in the database
    cursor.execute("""
        SELECT name FROM sqlite_master WHERE type='table' AND name='company_fundamentals'
    """)
    table_exists = cursor.fetchone()

    if table_exists:
        # If the table exists, first delete all data
        cursor.execute("DELETE FROM company_fundamentals")
        print("Existing data deleted from company_fundamentals.")
        # Update the table structure to add new fields if they do not exist
        update_table_structure(cursor)
    else:
        # If the table does not exist, create it with all required fields
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS company_fundamentals (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                symbol TEXT UNIQUE NOT NULL,
                pe_ratio REAL,
                market_cap INTEGER,
                revenue INTEGER,
                beta REAL,
                roa REAL,
                roe REAL,
                industry TEXT,
                dividend_yield REAL,
                dividend_per_share REAL,
                total_debt INTEGER,
                total_cash INTEGER,
                free_cash_flow INTEGER,
                operating_cash_flow INTEGER,
                net_income INTEGER,
                last_updated TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        """)
        print("Table company_fundamentals created.")

    conn.commit()
    conn.close()
    print("Database initialized: Table created/updated and old data deleted.")

# Call the function to initialize the database
create_database()


Existing data deleted from company_fundamentals.
Database initialized: Table created/updated and old data deleted.


In [37]:
import sqlite3  # Import the sqlite3 module for connecting to and working with SQLite databases
import random   # Import the random module for generating random numbers (not directly used in this code snippet, but may be used elsewhere)

def insert_into_db(fundamentals):
    """
    Insert the fundamental financial data into the SQLite database.
    
    :param fundamentals: A dictionary containing fundamental indicators for multiple stocks,
                         where the keys are the stock symbols and the values are sub-dictionaries
                         containing various metrics.
    """
    # Connect to the SQLite database named 'company_fundamentals.db'
    # If the database file does not exist, it will be created automatically.
    conn = sqlite3.connect('company_fundamentals.db')
    # Create a cursor object to execute SQL statements.
    cursor = conn.cursor()

    # Initialize an empty list to store the data to be inserted; each element is a tuple.
    data_to_insert = []

    # Iterate over each item in the fundamentals dictionary; symbol is the stock symbol,
    # and metrics is the corresponding indicator data.
    for symbol, metrics in fundamentals.items():
        # If the current stock's indicator data is empty, print a message and skip this stock.
        if metrics is None:
            print(f"⚠️ Skipping {symbol}: No data retrieved from API.")
            continue  # Skip to the next iteration if no data is available
        
        # Here, only the values corresponding to the new columns are added; other existing fields and logic remain unchanged.
        values = (
            symbol,
            metrics.get('P/E Ratio'),                # Price/Earnings Ratio
            metrics.get('Market Cap'),               # Market Capitalization
            metrics.get('Revenue'),                  # Revenue
            metrics.get('Beta'),                     # Beta
            metrics.get('Return on Assets (ROA)'),   # Return on Assets
            metrics.get('Return on Equity (ROE)'),   # Return on Equity
            metrics.get('industry'),                 # Industry
            metrics.get('dividendYield'),            # Dividend Yield
            metrics.get('dividend per share'),       # Dividend per Share
            metrics.get('total debt'),               # Total Debt
            metrics.get('total cash'),               # Total Cash
            metrics.get('free cash flow'),           # Free Cash Flow
            metrics.get('operating cash flow'),      # Operating Cash Flow
            metrics.get('net income'),               # Net Income
        )

        # Append the constructed tuple to the data_to_insert list.
        data_to_insert.append(values)

    # If there is data to insert, perform a bulk insert operation.
    if data_to_insert:
        try:
            # Execute the INSERT OR REPLACE statement with the new columns included.
            cursor.executemany("""
                INSERT OR REPLACE INTO company_fundamentals (
                    symbol,
                    pe_ratio,
                    market_cap,
                    revenue,
                    beta,
                    roa,
                    roe,
                    industry,
                    dividend_yield,
                    dividend_per_share,
                    total_debt,
                    total_cash,
                    free_cash_flow,
                    operating_cash_flow,
                    net_income
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """, data_to_insert)
        except sqlite3.IntegrityError as e:
            # If a data integrity error occurs during insertion, catch the exception and print an error message.
            print(f"❌ Database error: {e}")

    # Commit the transaction to write all changes to the database.
    conn.commit()
    # Close the database connection to release resources.
    conn.close()
    # Print a success message.
    print("✅ Database updated successfully.")


In [41]:
insert_into_db(fundamentals_data)

✅ Database updated successfully.


In [43]:
import sqlite3  # Import the sqlite3 module for working with SQLite databases

# Reconnect to the database
conn = sqlite3.connect('company_fundamentals.db')
# Obtain a cursor from the connection object to execute SQL statements
cursor = conn.cursor()

# Execute an SQL query to retrieve all records from the company_fundamentals table
cursor.execute("SELECT * FROM company_fundamentals")
# The fetchall() method returns all queried rows as a list, where each row is a tuple
rows = cursor.fetchall()

# Retrieve the table's column information (table structure)
# PRAGMA table_info(table_name) is used to query detailed information for each column in the table
cursor.execute("PRAGMA table_info(company_fundamentals)")
# The fetchall() method returns information for each column in the table, where each column is represented as a tuple
columns = cursor.fetchall()
# Extract the second element from each column tuple (i.e., the column name) to form a list of column names
column_names = [column[1] for column in columns]  # column[1] represents the column name in each column tuple

# Print the column names and all records
print("Columns:", column_names)
# Iterate over all rows and print each row
for row in rows:
    print(row)

# Close the database connection to release resources
conn.close()


Columns: ['id', 'symbol', 'pe_ratio', 'market_cap', 'revenue', 'beta', 'roa', 'roe', 'last_updated', 'industry', 'dividend_yield', 'dividend_per_share', 'total_debt', 'total_cash', 'free_cash_flow', 'operating_cash_flow', 'net_income']
(13, 'MMM', 17.038766355140186, 81630444000, 44.61692084241104, 1.017, 0.10467041236079061, 1.086153045288912, '2025-03-25 02:14:31', None, 0.02787514388327926, 0.73, 13659000000, 5600000000, 638000000, 1819000000, 4188000000)
(14, 'AOS', 18.79063869911919, 9544682291, 25.973949110692757, 1.177, 0.16469135802469137, 0.28330236262277675, '2025-03-25 02:14:31', None, 0.01898932734107241, 0.34, 216700000, 239600000, 473800000, 581800000, 533600000)
(15, 'ABT', 14.615158913744963, 219113988800, 24.224778349355955, 0.693, 0.1646154224089223, 0.2811765693185633, '2025-03-25 02:14:31', None, 0.019584182294331058, 0.59, 15021000000, 7616000000, 6351000000, 8558000000, 13402000000)
(16, 'ABBV', 73.32542459378213, 370741153500, 31.91263393759485, 0.598, 0.03165114